##**과제**: 수능 국어 문제 GPT-4로 풀어보기



Dataset Download: https://github.com/NomaDamas/KICE_slayer_AI_Korean

github의 data > 2023_11_KICE.json data를 colab으로 불러옵니다.

요구사항: https://www.notion.so/teamsparta/4-GPT-4-1712dc3ef514808cb792d55f5a09186e

Openai platform: https://platform.openai.com/docs/overview

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dajinfx/hanghaeAI2/blob/main/W4_2_GPT-4o.ipynb)



##Import

In [ ]:
import openai
import json
import requests
import numpy as np
import torch
import pandas as pd
from langchain.prompts import PromptTemplate

In [ ]:
gpt_model="gpt-4o"
temperature = 0.7  # 각 token을 샘플링할 때 사용하는 temperature 값입니다.
max_tokens = 4096  # 생성하는 최대 token 개수 입니다.
frequency_penalty = 0.0  # 같은 단어가 반복적으로 나오는 것을 방지하기 위한 옵션입니다.

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

##Gather GPT-4  API Info

Colab 에서 GPT API 파일 저장하고 가져오기 ( os 환경변수, 로컬경로 등 업로드가 다 않되는 상황에서 취한 방식)

In [ ]:
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
    print(f" {filename} is uploaded")

with open('OpenAI_APIKey.txt', 'r') as file:
    content = file.read()

Saving OpenAI_APIKey.txt to OpenAI_APIKey.txt
 OpenAI_APIKey.txt is uploaded


In [ ]:
print(content)

In [ ]:
openai.api_key = content

##함수: prediction(problem)

In [ ]:
#
def prediction(problem):
    completion = openai.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an assistant for solving Korean exam problems."},
            {"role": "user", "content": problem}
        ],
        model=gpt_model,
        response_format={"type": "json_object"},
    )
    return completion


##API access test

In [ ]:
# GPT 출력 테스트
problem_text = "Example question: What is the correct answer for this Korean problem?  output format: json"
result = prediction(problem_text)

if result:
    print(f"GPT-4 Prediction: {result}")
else:
    print("Failed to get a prediction.")

GPT-4 Prediction: ChatCompletion(id='chatcmpl-AnhXootNsihC8d2EvDkEC2KD2ARto', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"answer": "The correct answer for this Korean problem cannot be determined without the problem statement or choices provided. Please provide the full question for further assistance."}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736408820, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_5f20662549', usage=CompletionUsage(completion_tokens=33, prompt_tokens=39, total_tokens=72, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
response_content = result.choices[0].message.content
fruit_data = json.loads(response_content)

formatted_json = json.dumps(fruit_data, indent=4, ensure_ascii=False)
print(formatted_json)

{
    "answer": "The correct answer for this Korean problem cannot be determined without the problem statement or choices provided. Please provide the full question for further assistance."
}


##Data upload

In [ ]:
url = 'https://raw.githubusercontent.com/NomaDamas/KICE_slayer_AI_Korean/master/data/2023_11_KICE.json'
response = requests.get(url)
data = response.json()

In [ ]:
df = pd.DataFrame(data)
print(df.head())

                   id                                          paragraph  \
0    2023_11_KICE_1-3  사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 ...   
1    2023_11_KICE_4-9  (가)[A](중국에서 비롯된 유서(類書)는 고금의 서적에서 자료를 수집하고 항목별로...   
2  2023_11_KICE_10-13  법령의 조문은 대개 ‘A에 해당하면 B를 해야 한다.’처럼 요건과효과로 구성된 조건...   
3  2023_11_KICE_14-17  하루에 필요한 에너지의 양은 하루 동안의 총 열량 소모량인 대사량으로 구한다. 그중...   
4  2023_11_KICE_18-21  혼례를 마친 후 최척이 아내와 함께 장모를 모시고 집으로 돌아오매 하인들이 기뻐했다...   

   type                                           problems  
0     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
1     0  [{'question': '(가)와 (나)에 대한 설명으로 가장 적절한 것은?', ...  
2     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
3     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
4     1  [{'question': '윗글에 대한 설명으로 가장 적절한 것은?', 'choic...  


In [ ]:
dftest = df.copy()
print(dftest.head())

                   id                                          paragraph  \
0    2023_11_KICE_1-3  사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 ...   
1    2023_11_KICE_4-9  (가)[A](중국에서 비롯된 유서(類書)는 고금의 서적에서 자료를 수집하고 항목별로...   
2  2023_11_KICE_10-13  법령의 조문은 대개 ‘A에 해당하면 B를 해야 한다.’처럼 요건과효과로 구성된 조건...   
3  2023_11_KICE_14-17  하루에 필요한 에너지의 양은 하루 동안의 총 열량 소모량인 대사량으로 구한다. 그중...   
4  2023_11_KICE_18-21  혼례를 마친 후 최척이 아내와 함께 장모를 모시고 집으로 돌아오매 하인들이 기뻐했다...   

   type                                           problems  
0     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
1     0  [{'question': '(가)와 (나)에 대한 설명으로 가장 적절한 것은?', ...  
2     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
3     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
4     1  [{'question': '윗글에 대한 설명으로 가장 적절한 것은?', 'choic...  


In [ ]:
def count_probles_pro(problems):
    question_count = 0
    choices_count = 0
    answer_count = 0
    score_count = 0

    label_score = []

    count = 0
    for problem in problems:
        count += 1
        print(f"count: {count}")
        print(problem)
        print()
        if 'question' in problem:
            question_count += 1
        if 'question' in problem:
            question = problem['question']
            print(f"question: {question}")
        if 'choices' in problem:
            choices_count += 1
        if 'score' in problem:
            score_count += 1
            label_score.append(problem['score'])
            print(f"score: {problem['score']}")
        if 'answer' in problem:
            answer_count += 1

    return question_count,choices_count,answer_count,score_count,label_score

question_count,choices_count,answer_count,score_count,label_score = count_probles_pro(dftest['problems'][0])
#print(f"question:{question_count}, choices_count:{choices_count},answer_count: {answer_count}, score_count: {score_count}")

##데이터 정답 저장

학습결과를 비교하기 위한 조치,  list 로 저장하여 데이터 순서와 정확성 문제 없도록 저장합니다.

In [ ]:
label_answer = []
def get_label_answer(problems):
    answer_count = 0

    count = 0
    for problem in problems:
        count += 1
        if 'answer' in problem:
            answer_count += 1
            label_answer.append(problem['answer'])
            #print(f"answer: {problem['answer']}")
    return label_answer

for index in range(len(dftest['problems'])):
  label_answer = get_label_answer(dftest['problems'][index])


In [ ]:
print(label_answer)
print(len(label_answer))

[4, 5, 1, 4, 5, 3, 2, 5, 2, 4, 5, 2, 5, 3, 4, 4, 1, 4, 3, 3, 5, 1, 5, 3, 3, 4, 1, 5, 1, 2, 1, 4, 2, 3, 1, 5, 3, 4, 4, 3, 2, 4, 5, 2, 1]
45


In [ ]:
len(df['problems'])

11

In [ ]:
label = []
def remove_answer_and_score(problems):
    for problem in problems:
        if 'answer' in problem:
            del problem['answer']  # remove answer
            print("")
        if 'score' in problem:
            label.append(problem['score'])
            print(problem['score'])
            #del problem['score']  # remove score
    return problems

#print(dftest.head())

In [ ]:
dftest['problems'] = dftest['problems'].apply(remove_answer_and_score)

In [ ]:
question_count,choices_count,answer_count,score_count = count_probles_pro(dftest['problems'][0])
print(f"question:{question_count}, choices_count:{choices_count},answer_count: {answer_count}, score_count: {score_count}")

count: 1
{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices': ['같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다.', '다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다', '독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다.', '독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.', '독자는 책을 읽을 때 자신이 속한 사회나 시대의 영향을 받으며 필자와 간접적으로 대화한다']}

question: 윗글의 내용과 일치하지 않는 것은?
count: 2
{'question': '다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내용으로 가장 적절한 것은?', 'question_plus': "ⓐ('음악 시간에 들었던 베토벤의 교향곡 <합창>이 위대한 작품인 이유는 무엇일까?'하는 생각)에, 베토벤에 대한 책을 빌렸다. 책에서는 기약만으로 구성됐던 교향곡에 성악을 결합헤 개성을 드러냈다는 점에서 ⓑ(이 곡이 낭만주의 음악의 특징을 보여 준다고 했다.) <합창>을 해설한 부분에 이어, 베토벤의 생애에 관한 뒷부분도 읽었는데, ⓒ(이 내용들을 종합해, 절망적 상황에서도 열정적으로 자신이 좋아하는 일을 했기에 교향곡 구성의 새로움을 보여 준 명작이 탄생했음을 알게 됐다.) 이후 ⓓ(내가 진정으로 좋아하는 일이 무엇인지 나에게 묻게 되었다.) ⓔ(글 쓰는 일에서 가장 큰 행복을 느꼈던 나를 발견)할 수 있었고, 나도 어떤 상황에서든 좋아하는 일을 계속해야겠다고 생각했다.", 'choices': ['ⓐ와 ⓑ에는 모두 ‘독자 자신에게서 답을 찾는 질문’이 나타난다.', 'ⓒ와 ⓓ에는 모두 ‘책에 명시된 내용’에서 질문의 답을 찾아내는 모습이 나타난다.', 'ⓐ에는 ‘책에서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다.', 'ⓑ에는 ‘책에서 답을 찾는 질문’이, ⓒ에는 그에 대

##데이터 전처리

In [ ]:
def question_deliver(dt):
    count = 0
    #print(dt['paragraph'][5])
    sample = []
    for item in range(len(dt)):
      #count +=1
      #print("id: \n",item['id'])
      #print(type(item))
      paragraph = dt['paragraph'][count]
      #print("paragraph: \n",paragraph)
      #print(dt['problems'][count])

      sample_datas = []
      for problem in dt['problems'][count]:
        choices_q = []

        question1 = problem['question']
        if 'question_plus' in problem:
          question2_plus = problem['question_plus']
        else:
          question2_plus=""

        choices_count = 0
        for choice in problem['choices']:
          #str_choice = str(choices_count)
          #print(str_choice)
          choices_q.append(choice)
          choices_count +=1

        sample_data = {
            "question1": question1,
            "question2_plus": question2_plus,
        }
        #print("question1-----------: ",question1)

        for i,choice__ in enumerate(choices_q, 1):
          sample_data[f"choices{i}"] = choice__

        sample_datas.append(sample_data)

      problem_data={
          "paragraph": paragraph,
          "problems": sample_datas
      }

     #print(problem_data)
      sample.append(problem_data)
      count += 1
      #print("count: ",count)
    return sample


##데이터 전처리 포멧 검사

GPT-4에 쉽게 넘기기 위한 전처리 데이터 포멧으로 전환함.

In [ ]:
question = question_deliver(df)

responses = []
for q in question:
  sample_string = json.dumps(q, ensure_ascii=False, indent=2)
  print(sample_string)
  responses.append(sample_string)
  break

{
  "paragraph": "사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도 책을 통한 소통의 즐거움을 나누는 일이다.",
 

##Prediction method

In [ ]:
def predictions(prompt,data):
    completion = openai.chat.completions.create(
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": data}
        ],
        model=gpt_model,
        #response_format={"type": "json_object"},
        temperature=temperature
    )
    return completion

##Prompt

In [ ]:
basic_prompt = """
    국어 시험 문제를 푸는 똑똑한 학생으로서 다음 문제의 답을 구하세요.
    지문을 읽고, 질문에 대한 답을 1부터 5까지의 선택지 중에 한 개만 골라서 대답해야 합니다. 대답은 다 하되 json 형태로,  마지막 정답은 숫자로만 주세요

    지문 :
    {paragraph}

    질문 :
    {question1}

    <보기> :
    {question2_plus}

    선택지 :
    1번 - {choices1}
    2번 - {choices2}
    3번 - {choices3}
    4번 - {choices4}
    5번 - {choices5}

    1번, 2번, 3번, 4번, 5번 중에 하나를 정답으로 고르세요. 정답 :


    정담을 아래포멧으로  정래히주세요:

    "problem": [
      {
        "question": "윗글의 내용과 일치하지 않는 것은?",
        "answer": 4
      },
      {
        "question": "다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내용으로 가장 적절한 것은?",
        "answer": 4
      },
      {
        "question": "윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은?",
        "answer": 1
      }
    ]
    """


##Taining start

In [ ]:
gpt_result = []
pred = []
count = 0
for q in responses:
  #print(q)
  sample_string = json.dumps(q, ensure_ascii=False, indent=2)
  #print(sample_string)
  print("--------------------------------- ",count)
  response = predictions(basic_prompt,sample_string)
  content = response.choices[0].message.content
  gpt_result.append(response)
  #print(content)


  try:
        if content.startswith("```json"):
            content = content.replace("```json", "").replace("```", "").strip()
        content_dict = json.loads(content)


        if "problem" in content_dict:
            for problem in content_dict["problem"]:
                pred.append(problem["answer"])
  except json.JSONDecodeError as e:
        print("JSON 해제실패:", e)
        print("content :", content)

  count+=1
  #break



---------------------------------  0
```json
{
  "problem": [
    {
      "question": "윗글의 내용과 일치하지 않는 것은?",
      "answer": 4
    },
    {
      "question": "다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내용으로 가장 적절한 것은?",
      "answer": 4
    },
    {
      "question": "윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은?",
      "answer": 1
    }
  ]
}
```
---------------------------------  1
```json
{
  "problem": [
    {
      "question": "(가)와 (나)에 대한 설명으로 가장 적절한 것은?",
      "answer": 4
    },
    {
      "question": "[A]에 대한 이해로 적절하지 않은 것은?",
      "answer": 5
    },
    {
      "question": "㉮에 대한 이해를 바탕으로 ㉠, ㉡에 대해 파악한 내용으로 적절하지 않은 것은?",
      "answer": 3
    },
    {
      "question": "㉯를 반박하기 위한 ‘이수광’의 말로 가장 적절한 것은?",
      "answer": 2
    },
    {
      "question": "(가), (나)를 읽은 학생이 <보기>의 임원경제지에 대해 보인 반응으로 적절하지 않은 것은?",
      "answer": 5
    },
    {
      "question": "문맥상 ⓐ～ⓔ와 바꾸어 쓰기에 적절하지 않은 것은?",
      "answer": 2
    }
  ]
}
```
---------------------------------  2
```json
{
  "problem": [
  

In [ ]:
print(pred)
print(len(pred))

[4, 4, 1, 4, 5, 3, 2, 5, 2, 4, 5, 2, 5, 3, 3, 4, 1, 4, 3, 3, 5, 1, 5, 3, 3, 4, 1, 5, 1, 2, 1, 4, 2, 3, 1, 5, 3, 4, 4, 1, 2, 4, 5, 4, 1]
45


##정확도 함수

In [ ]:
def accuracy(label_score, pred):
    total_samples = 0
    correct_predictions = 0

    for label, prediction in zip(label_score, pred):
        if label == prediction:
            correct_predictions += 1
        total_samples += 1
    print("total_samples: ",total_samples)
    print("correct_predictions: ",correct_predictions)
    return correct_predictions / total_samples

In [ ]:
acc = accuracy(label_score, pred)
print(acc)

total_samples:  45
correct_predictions:  41
0.9111111111111111


##결론

정확도 91% 로 나온 결과로 만족합니다.

여러가지 프롬프트,  rag 기반으로 시도 욕망.